# Map Reduce

Eerst maken we een aparte directory aan voor alles wat we voor deze notebook gaan gebruiken in hdfs. Dit om conflicten of het overschrijven van gegevens te vermijden.


In [ ]:
from hdfs import InsecureClient

map = 'MapReduce'

client = InsecureClient('http://localhost:9870', user='bigdata')

if client.status(map, strict=False) is None:
    client.makedirs(map)
else:
    # do some cleaning in case anything else than *.txt is present
    for f in client.list(map):
        client.delete(map + '/' + f, recursive=True)

client.upload(map, 'input.txt')
client.upload(map, 'titanic.csv')
with client.read(map + '/input.txt') as reader:
    content = reader.read()
    print(content.decode('utf-8'))

## Wat is MapReduce

MapReduce is een programmeermodel om eenvoudig distributed data te verwerken.
Het is belangrijk om te realiseren dat de programma's die je hier schrijft een parallel uitgevoerd worden op verschillende stukjes data (De map-fase) om daarna in de reduce-fase tot een finale output teruggebracht te worden.
Er gebeuren 5 stappen bij het uitvoeren van een MapReduce programma
* Bepalen op welke nodes de code uitgevoerd wordt (wordt door YARN gedaan afhankelijk van de locatie van de blocks)
* Uitvoeren van de Map-code (Geschreven door de developer naar eigen wens)
* Shuffle, ouput van de map-fase doorsturen naar andere nodes die de resultaten gaan reduceren (Automatisch)
* uitvoeren van de Reduce-code (Geschreven door de developer naar eigen wens)
* Combineren van de reduce output tot 1 gehele/finale output (Automatisch)

Uit bovenstaand stappenplan is het duidelijk dat er twee zaken moeten geimplementeerd worden bij het schrijven van een MapReduce toepassing.
Echter zullen we eerst een aantal voorbeelden bestuderen met reeds bestaande implementaties om zo meer vertrouwd te geraken met de flow van MapReduce.

## Voorbeelden van bestaande applicaties

Reeds een aantal default MapReduce applications zijn mee geinstalleerd met Hadoop.
De jar die deze toepassingen bevat kan gevonden worden in hadoop/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.3.0.jar.
Wanneer je deze jar uitvoert met onderstaande commando krijg je een lijst met de beschikbare applicaties

In [ ]:
!hadoop jar hadoop-mapreduce-examples-3.3.6.jar

In deze notebook gaan we vooral focussen op het typische probleem van wordcount.
Dit is een toepassing dat gaat tellen hoe vaak elk woord voorkomt in een bepaalde tekst.
Om meer informatie over deze toepassing te krijgen kan je gebruik maken van het volgende commando:

In [ ]:
!hadoop jar hadoop-mapreduce-examples-3.3.6.jar wordcount

In de eerste cell hebben we het input.txt bestand geupload. Of dit correct gebeurd is kan je controleren op de [file explorer van het hdfs](http://localhost:9870/explorer.html#/user/bigdata)

Met onderstaande commando kan nu het precompiled word count example uitgevoerd worden

In [ ]:
!hadoop jar hadoop-mapreduce-examples-3.3.6.jar wordcount /user/bigdata/MapReduce/input.txt /user/bigdata/MapReduce/output

Na het uitvoeren van het wordcount applicatie is er op het HDFS een extra folder toegevoegd met als naam **output**.
De plaats waar de output bewaard wordt is opgegeven in het bovenstaande commando en kan bekeken worden via de file-explorer van het hdfs.
Deze folder bevat de volgende files:
* Een _SUCCESS file dat leeg is. Deze file wordt gebruikt om aan te geven dat de MapReduce applicatie die resulteerde in de output goed afgerond was.
* Een part-r-00000 file dat de output bevat. Indien de output te groot is kan het zijn dat deze file in meerdere files gesplitst wordt. Hier is dit niet het geval en bevat deze file alle ouput. De output van het bovenstaande commando zijn key-value paren waar de keys de verschillende woorden zijn en de values hoeveel keer elk woord voorkomt.

De output van het commando kan dan met onderstaande code uitgelezen worden:

In [ ]:
with client.read(map + '/output/part-r-00000') as reader:
    content = reader.read()
    print(content.decode('utf-8'))

## Logs/process monitoring

Bovestaande wordcount applicatie wordt uitgevoerd op YARN op de verschillende nodes van de cluster. Dit houdt in dat YARN gebruikt kan worden om de status van de applicatie op te volgen. Hiervoork kan je server naar [de webpagina van Yarn](http://localhost:8088). Na het klikken op de juiste status krijg je de correcte applicatie te zien. Hier kan je ook de logs bekijken door op logs te klikken.

**LET OP: Doordat we in de webbrowser buiten de docker-containers zitten, kunnen we niet de historyserver hostname gebruiken. Als je een DNS-lookup error krijgt, vervang dan de domeinnaam/hostname door localhost.**

## OEFENING: MAPREDUCE 

Probeer nu op basis van bovenstaande reeds bestaande applicaties de gemiddelde lengte van de woorden in de text te bepalen.

In [ ]:
# uit te voeren op commando voor de gemiddelde lengte van de woorden te bekomen

De resource manager houdt ook een overzicht bij van de uitgevoerde applicaties, hun status, runtime en eventuele loggings. Na bovenstaande commando's uit te voeren moet je iets gelijkaardigs zien als in de onderstaande screenshot.

![yarn of mapreduce](images/yarn_001.png)

## Zelf implementeren van MapReduce applicaties

Natuurlijk zijn er veel meer zaken mogelijk om te berekenen met map-reduce toepassingen dan de reeds gecompileerde in hadoop.
Zoals eerder aangehaald valt vooral het coderen van de Map- en Reducestap hierbij op de schouders van de developer.
De standaard programmeertaal van MapReduce is Java en dus ook het grootste deel van de documentatie over MapReduce is geschreven met behulp van Java.
Deze programmas moeten dan gecompileerd worden tot een jar dat geupload kan worden naar de overeenkomstige nodes en daar uitgevoerd.
De api overview van hadoop kan je [hier](https://hadoop.apache.org/docs/stable/api/org/apache/hadoop/mapred/package-summary.html) vinden.

De code voor het wordcount example ziet er als volgt uit:

In [ ]:
%%file WordCount.java
// dit schrijft onderstaande cell naar een file in de huidige directory
import java.io.IOException;
import java.util.StringTokenizer;

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

public class WordCount {

   
}

Deze code bevat drie delen:
* De main() functie: verzorgt de configuratie van de uit te voeren taak. Geeft aan wat de Map en Reduce klassen zijn, wat de input is, hoe de output bewaard wordt ,...
* De Map-klasse met de map() functie bevat de code voor de mapping-fase
* De Reduce-klasse met de reduce() functie bevat de code voor de reduce-fase

De laatste twee klassen zijn hier gecodeerd als geneste klassen. Deze hadden ook in aparte files geplaatst kunnen worden.
Nu moet deze code eerst omgezet/gecompileerd worden tot een .jar file. Deze kan dan analoog uitgevoerd worden als hierboven met het voorbeeldcode.
Deze twee stappen kunnen uitgevoerd worden door onderstaande commando's.

In [ ]:
# compileren tot .jar
!javac -cp "hadoop-common-3.3.6.jar:hadoop-mapreduce-client-core-3.3.6.jar" -d . WordCount.java
!jar cvf wordcounter.jar *.class

In [ ]:
# execute
!hadoop jar wordcounter.jar WordCount /user/bigdata/MapReduce/input.txt /user/bigdata/MapReduce/output_java

## Implementeren via Python

Hoewel het Hadoop Ecosysteem geprogrammeerd is in Java en het dus er goed mee samenwerkt, is het niet verplicht om Java te gebruiken om MapReduce applicaties te schrijven.
Java krijgt namelijk veel kritiek, vooral doordat er veel code nodig is om eenvoudige zaken te programmeren.
Om andere programmeertalen te gebruiken worden er verscheidene API's aangeboden door Hadoop, namelijk
* Hadoop Streaming
    * Communicatie via stdin/stdout
    * Gebruikt door hadoopy, mrjob, ...
* Hadoop Pipes
    * C++ interface voor Hadoop
    * Communicatie via sockets
    * Gebruikt door pydoop


### Mrjob

De eerste API die we bekijken is MrJobs.
Deze API maakt gebruik van de Hadoop Streaming API.
De voordelen van MrJobs zijn:
* Uitgebreidde documentatie
* Code kan lokaal uitgevoerd worden als test
* Data serialisatie gebeurt automatisch (nadeel van Streaming API)
* Werkt met Amazon Elastic MapReduce en Google Cloud Dataproc

Het grootste nadeel is dat de StreamingAPI niet de volledige kracht heeft van het Hadoop ecosysteem omdat alles omgezet wordt naar strings (jsons)

Installatie van deze package gebeurt als volgt:

In [ ]:
%%file wordcount_mrjob.py


Deze code kan uitgevoerd worden door het commando hieronder.
Er zijn twee belangrijke parameters om toe te voegen aan het commando.
* -r: Deze parameter geeft aan welke file uitgelezen wordt. Dit kan een lokale file zijn of een file op het hdfs. Om een hdfs-file aan te spreken gebruik je de volgende structuur: hdfs:///{path to file}
* -o: Deze parameter geeft aan waar de data moet bewaard worden. Als deze parameter ontbreekt wordt het in de stdout geprint. Met de parameter kan je aangeven in welke file (lokaal of hdfs) de output moet bewaard worden.

Meer informatie over hoe mrjob applicaties gestart kunnen worden vind je in [de documentatie](https://mrjob.readthedocs.io/en/latest/guides/quickstart.html#running-your-job-different-ways)

De bovenstaande manier vereist echter dat je steeds een python-file aanmaakt die dan via commandline gestart wordt.
De reden hiervoor is is dat **de file naar de cluster verstuurd wordt**.
Om de output te bekijken kan het output.txt bestand gedownload en uitgeprint worden als volgt:

In [ ]:
with client.read(map + '/output_txt/part-00000') as reader:
    content = reader.read()
    print(content.decode('utf-8'))

## Oefeningen

Maak de nodige mapreduce applicaties voor de volgende zaken te berekenen (je moet de output niet bewaren in een file):
* De gemiddelde woordlengte
* Het aantal keer dat elk karakter voorkomt
* Het aantal woorden dat begint met elke letter
* Het aantal woorden in de tekst

Als een applicatie crasht, kan het zijn dat ze op de cluster nog een hele tijd actief blijft en het starten van nieuwe applicaties kan tegenhouden.
Met onderstaande commando kan je een bestande applicatie afsluiten/killen

In [ ]:
# killen van een applicatie
yarn application -kill <Application_ID>
# appliation id kan je vinden via de webbrowser van yarn (localhost:8088)

In [ ]:
%%file vraag1.py
# vraag 1: gemiddelde woordlengte


In [ ]:
# test vraag 1

In [ ]:
%%file vraag2.py
# vraag 2: het aantal keer dat elk karakter voorkomt


In [ ]:
# test vraag 2

In [ ]:
%%file vraag3.py
# vraag 3: * Het aantal woorden dat begint met elke letter


In [ ]:
# test vraag 3

In [ ]:
%%file vraag4.py
# vraag 4: Het aantal woorden in de tekst


In [ ]:
# test vraag 4

## Werken met MapReduce voor gestructureerde data

Mapreduce kan ook gebruikt worden om te werken met gestructureerde data (bijvoorbeeld een csv) waar elke rij 1 data-element voorsteld.
Het is hier echter wel belangrijk dat alle data op 1 lijn gecombineerd wordt dus multiline csv's, jsons of xml bestanden kunnen niet verwerkt worden.

Onderstaande code is een voorbeeld voor hoe je een csv kan uitlezen en een aantal statistieken kan berekenen. Hierin leer je vooral:
* Hoe de csv lijn per lijn te verwerken en kolommen te detecteren
* Hoe de header rij eruit filteren
* Hoe meerdere berekeningen op een iterator te doen

We gebruiken hiervoor de titanic.csv file. Let op dat die naam van de passagier hierbij komma's kan bevatten dus dit vereist wat extra aandacht.
Daarnaast gebruiken we de lokale versie van het bestand en niet de geuploadde versie.

We willen de volgende zaken berekenen door middel van 1 map-reduce applicatie:
* Gemiddelde leeftijd
* Percentage overleeft
* Percentage mannelijke passagiers
* Percentage vrouwelijke passagiers die het overleefd hebben: 30% betekend dat 30% van de vrouwelijke passagiers het overleefd hebben

In [ ]:
%%file structured.py

In [ ]:
# test structured

## Debugging

Indien er een python script voor een map-reduce applicatie een error bevat is het niet altijd eenvoudig om de specifieke foutmelding te vinden. Dit is vooral merkbaar als we een runtime errors.
Indien we in het voorgaande script de beveiligingen rond het casten van empty strings verwijderen. Dan krijgen we foutmeldingen. Indien we ditzelfde nu uitvoeren (maar uitvoeren op de cluster in plaats van lokaal in tegenstelling tot het voorgaande commando), dan krijgen we geen duidelijke foutmelding.

In [ ]:
%%file structured_error.py

Na het uitvoeren van bovenstaande script, dan zie je dat de applicatie gestart wordt (map 0% reduce 0%) maar een duideljke foutmelding is er niet (run time exception).
Om de logs te bekijken kan je 
* surfen gaan naar de historyserver die beschikbaar is op de url: localhost:8188
* het volgende command-line commando uitvoeren. Dit haalt alle std-err logs van alle containers op. Zoek naar een python error en je zal vinden wat er misgaat.
* een python script schrijven om de yarn log-server te bevragen

In [ ]:
!yarn logs -applicationId {app id} | sed -n '/LogType:stderr/,/End of LogType:stderr/p'

## Afsluitende opmerkingen

Het is belangrijk om te beseffen dat deze applicatie op meerdere nodes kan tegelijkertijd uitgevoerd worden. 
Dit heeft als gevolg dat je geen persistente counters kan toevoegen in de reducer om gemiddelden en dergelijke te berekenen.
Een globaal overzicht van het dataframe kan maar **in de mapper** behaald worden.

Daarnaast is er nog een andere stap mogelijk dan mapper of reducer. Dat is **de combiner** stap. Dit is een stap die runt per node en gebruikt kan worden om al wat combinaties te doen zodat er minder data tussen nodes moet verstuurd worden. In grote applicaties/datasets kan dit heel wat internettrafiek besparen wat de werking van de cluster ten goede komt.

Ten derde is het ook mogelijk om meerdere stappen te definieren in de mrjob applicatie. Dit kan door de steps functie te implementeren en daar elke stap in te definieren. Elke stap hierin kan beschikken over een eigen mapper/reducer en combiner functie. Meer informatie vind je [in de documentatie](https://mrjob.readthedocs.io/en/latest/guides/quickstart.html#writing-your-second-job)

## Verdere oefeningen

Gebruik nu de titanic csv om met behulp van een mapreduce applicatie de volgende zaken te berekenen:
* Het aantal unieke plaatsen waar personen aan boord zijn gekomen (embarked kolom)
* Het aantal ontbrekende waarden in de Cabin kolom
* De volgende statistische waarden voor de ticketprijs (Fare) kolom: min, max, mean, std
* De langste naam van een passagier
* Het aantal passagiers per klasse
* Het totaal aantal passagiers op de titanic

In [ ]:
%%file structured_oefening.py
